In [ ]:
# Import libraries
import os
import requests
from dotenv import load_dotenv
import datetime
from twilio.rest import Client

In [ ]:
load_dotenv()

# Retrieve APIs keys from environment variables
ALPHA_API_KEY = os.environ.get('ALPHA_API_KEY')
NEWS_API_KEY = os.environ.get('NEWS_API_KEY')
TWILIO_ACCOUNT_SID = os.environ.get('TWILIO_ACCOUNT_SID')
TWILIO_AUTH_TOKEN = os.environ.get('TWILIO_AUTH_TOKEN')
FROM_WPP = os.environ.get('FROM_WPP')
TO_WPP = os.environ.get('TO_WPP')
ALPHA_URL = os.environ.get('ALPHA_URL')
ALPHA_SYMBOL_SEARCH_URL = os.environ.get('ALPHA_SYMBOL_SEARCH_URL')
NEWS_URL = os.environ.get('NEWS_URL')

In [ ]:
def get_symbol(keyword: str) -> list:
    url = ALPHA_SYMBOL_SEARCH_URL
    params = {
        'function': 'SYMBOL_SEARCH',
        'keywords': keyword,
        'apikey': ALPHA_API_KEY
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    
    raw_data = response.json()['bestMatches']
    
    filtered_data = [
        {
            'symbol': stock['1. symbol'],
            'name': stock['2. name'],
            'region': stock['4. region'],
            'currency': stock['8. currency']    
        }
        for stock in raw_data
    ]
    return filtered_data

In [ ]:
def stock_selection(stock_list: list) -> str | None:
    
    if not stock_list:  # Check if the list is empty
        print("No stocks available.")
        return None

    # Display available stock options
    print("Available stocks:")
    
    for index, stock in enumerate(stock_list, start=1):
        print(f"{index}. {stock['symbol']}: {stock['name']} ({stock['region']}, {stock['currency']})")
        
    while True: 
        try:
            choice = int(input("Select a stock by entering the corresponding number: ")) - 1
            if 0 <= choice < len(stock_list):
                selected_stock = stock_list[choice]
                company = selected_stock['name'].split(" ")[0]
                break
            else:
                print("Invalid selection. Please enter a valid number.")
        except ValueError:
            print("Invalid input. Please enter a number.")
    print(f"You selected: {selected_stock['symbol']}: {selected_stock['name']} ({selected_stock['region']}, {selected_stock['currency']})")
    return selected_stock['symbol'], company

In [ ]:
# Stock variation calculation
def get_stock_variation(stock_symbol: str) -> bool:
    """Fetch stock variation percentage from the Alphavantage API and check if it exceeds 5%."""
    # Set up Time Series Daily API request to get daily time series data for the chosen stock
    alpha_params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': stock_symbol.upper(),
        'apikey': ALPHA_API_KEY
    }
    
    # Send request and handle response
    response = requests.get(ALPHA_URL, params=alpha_params)
    response.raise_for_status()
    
    # Parse stock time series data
    raw_data = response.json()
    data = raw_data["Time Series (Daily)"]
    dates = sorted(data.keys(), reverse=True) # Get sorted available dates from API request
    
    try:
        yesterday_date = dates[0]
        two_days_ago_date = dates[1]
    except IndexError:
        print("Not enough data to calculate stock variation")
        return False
    
    # Extract closing prices for the last two days
    yesterday_close = float(data[yesterday_date]["4. close"])
    two_days_close = float(data[two_days_ago_date]["4. close"])
    
    # Calculate the percentage difference in closing prices
    percentage_difference = round(((yesterday_close - two_days_close) / two_days_close * 100),2)
    
    print(f'Stock price variation:{percentage_difference}%')
    
    # Determine if the stock price variation is within 3%
    if abs(percentage_difference) >= 3:
        print("The variation is more than 3%")
        return True
    else:
        print("The variation is within 3%")
        return False

In [ ]:
# Fetch news articles
def get_news(query,lang='en') -> list:
    """Retrieve top 3 news articles based on the query."""
    
    yesterday_date = datetime.datetime.now() - datetime.timedelta(days=1)
    
    # Set up API request parameters
    news_params = {
        'apiKey': os.environ.get('NEWS_API_KEY'),
        'qInTitle': query,
        'from': yesterday_date,
        'language': lang,
        'sortBy':'relevancy'
    }
    
    # Send request to News API
    response = requests.get(NEWS_URL, news_params)
    response.raise_for_status()
    data = response.json()
    
    return data['articles'][0:3] # Return top 3 news articles

In [ ]:
# Send wpp message
def send_wpp_message(message):
    """Send a WhatsApp message using Twilio."""
    client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
    msn = client.messages.create( 
        body=message,
        from_=FROM_WPP,
        to=TO_WPP,
        )
    return msn.status

In [ ]:
def main():
    
    # Get user input for stock keyword search
    stock_keyword = input('Enter a keyword to search for stocks (e.g., company name or ticker symbol):')
    stock_options = get_symbol(stock_keyword)
    
    stock_symbol, company_name = stock_selection(stock_options)
    
    stock_price_variation = get_stock_variation(stock_symbol)
    news = get_news(company_name)
    
    if stock_price_variation:
        print("Stock variation detected. Fetching news articles...")
        print(news)
        
        for article in news:
            message = f"Headline: {article['title']}\nBrief: {article['description']}\nLink: {article['url']}"
            send_wpp_message(message)
            print(f"Message sent: {message}")
    else:
        print("No stock variation detected")
        
        

In [ ]:
main()